# Segmenting and Clustering Neighborhoods

## Import and Cleaning

1. The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
2. Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
3. More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.
4. If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park.
5. Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making.
6. In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.
7. Submit a link to your Notebook on your Github repository. (10 marks)

In [1]:
#Imports
import numpy as np
import pandas as pd

In [2]:
#Import All Data
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
imports = pd.read_html(url, header=0)
df = pd.DataFrame(imports[0])
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [3]:
df.describe()

,Postcode,Borough,Neighbourhood
count,288,288,288
unique,180,12,209
top,M8Y,Not assigned,Not assigned
freq,8,77,78


In [4]:
# delete "Not assigned" and rename columns
df = df.set_index("Borough")
df = df.drop("Not assigned", axis=0)
df.rename(columns={"Postcode": "PostalCode"}, inplace=True)
df=df.reset_index()
df.head()

,Borough,PostalCode,Neighbourhood
0,North York,M3A,Parkwoods
1,North York,M4A,Victoria Village
2,Downtown Toronto,M5A,Harbourfront
3,Downtown Toronto,M5A,Regent Park
4,North York,M6A,Lawrence Heights


In [5]:
df.shape

(211, 3)

In [6]:
#combine rows
dfN=pd.DataFrame(df.groupby(["PostalCode","Borough"])['Neighbourhood'].apply(lambda tags: ','.join(tags)))
dfN=dfN.reset_index()

In [7]:
for index, row in dfN.iterrows():
    x=row['Neighbourhood']
    y=row['Borough']
    z=row['PostalCode']
    if x=="Not assigned":
        print(index,z,y,x)
        dfN.at[index,"Neighbourhood"]=y

85 M7A Queen's Park Not assigned


In [8]:
dfN.tail(20)

,PostalCode,Borough,Neighbourhood
83,M6R,West Toronto,"Parkdale,Roncesvalles"
84,M6S,West Toronto,"Runnymede,Swansea"
85,M7A,Queen's Park,Queen's Park
86,M7R,Mississauga,Canada Post Gateway Processing Centre
87,M7Y,East Toronto,Business Reply Mail Processing Centre 969 Eastern
88,M8V,Etobicoke,"Humber Bay Shores,Mimico South,New Toronto"
89,M8W,Etobicoke,"Alderwood,Long Branch"
90,M8X,Etobicoke,"The Kingsway,Montgomery Road,Old Mill North"
91,M8Y,Etobicoke,"Humber Bay,King's Mill Park,Kingsway Park Sout..."
92,M8Z,Etobicoke,"Kingsway Park South West,Mimico NW,The Queensw..."


In [9]:
dfN.shape

(103, 3)

## Adding coordinates

1. get the latitude and the longitude coordinates of each neighborhood from https://geocoder.readthedocs.io/index.html.
2. Once you are able to create the above dataframe, submit a link to the new Notebook on your Github repository.

In [11]:
!wget -O data.csv http://cocl.us/Geospatial_data

--2019-07-09 05:03:02--  http://cocl.us/Geospatial_data
Resolving cocl.us (cocl.us)... 159.8.72.228
Connecting to cocl.us (cocl.us)|159.8.72.228|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://cocl.us/Geospatial_data [following]
--2019-07-09 05:03:03--  https://cocl.us/Geospatial_data
Connecting to cocl.us (cocl.us)|159.8.72.228|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2019-07-09 05:03:03--  https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv
Resolving ibm.box.com (ibm.box.com)... 185.235.236.197
Connecting to ibm.box.com (ibm.box.com)|185.235.236.197|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /public/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2019-07-09 05:03:03--  https://ibm.box.com/public/static/9afzr83pps4pwf2smjjcf1y5

In [12]:
data = pd.read_csv("data.csv") 
data.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [15]:
dfTotal=dfN.merge(data, left_on='PostalCode', right_on='Postal Code')
dfTotal.drop(["Postal Code"], axis=1, inplace=True)
dfTotal.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


1. Explore and cluster the neighborhoods in Toronto. You can decide to work with only boroughs that contain the word Toronto and then replicate the same analysis we did to the New York City data. It is up to you.
2. to add enough Markdown cells to explain what you decided to do and to report any observations you make.
3. to generate maps to visualize your neighborhoods and how they cluster together.
4. Once you are happy with your analysis, submit a link to the new Notebook on your Github repository

In [ ]:
# The code was removed by Watson Studio for sharing.